In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [38]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [39]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')

data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [40]:
n_steps = 5
x_data, y_data = split_sequence(data, n_steps)

In [41]:
x_data = x_data.reshape((x_data.shape[0], -1))
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data.shape

(421, 25)

In [42]:
final_predict_data = x_data[-1:,:].copy()

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=0.9, random_state=66, shuffle=True
)

In [43]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_data.shape[1],)))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

# # 훈련
# model.compile(loss='mse', optimizer='adam', metrics=['acc'])
# model.fit(x, y, epochs=100, batch_size=1, validation_split=0.1)

In [44]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=50, mode='auto')
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=1000, validation_split=0.1,
         callbacks=[early_stopping])

Train on 340 samples, validate on 38 samples
Epoch 1/1000
340/340 [==============================] - 0s 1ms/step - loss: 2193014657.5059 - acc: 0.0000e+00 - val_loss: 2128091567.1579 - val_acc: 0.0000e+00
Epoch 2/1000
340/340 [==============================] - 0s 97us/step - loss: 2166875676.6118 - acc: 0.0000e+00 - val_loss: 2044763230.3158 - val_acc: 0.0000e+00
Epoch 3/1000
340/340 [==============================] - 0s 98us/step - loss: 1956242441.0353 - acc: 0.0000e+00 - val_loss: 1453063376.8421 - val_acc: 0.0000e+00
Epoch 4/1000
340/340 [==============================] - 0s 100us/step - loss: 1078894477.5529 - acc: 0.0000e+00 - val_loss: 989724163.3684 - val_acc: 0.0000e+00
Epoch 5/1000
340/340 [==============================] - 0s 94us/step - loss: 787399939.7647 - acc: 0.0000e+00 - val_loss: 638757658.9474 - val_acc: 0.0000e+00
Epoch 6/1000
340/340 [==============================] - 0s 99us/step - loss: 646101120.0000 - acc: 0.0000e+00 - val_loss: 616160591.1579 - val_acc: 0.000

340/340 [==============================] - 0s 72us/step - loss: 2965464.3118 - acc: 0.0000e+00 - val_loss: 1957269.8421 - val_acc: 0.0000e+00
Epoch 53/1000
340/340 [==============================] - 0s 79us/step - loss: 2659314.1912 - acc: 0.0000e+00 - val_loss: 1408865.9918 - val_acc: 0.0000e+00
Epoch 54/1000
340/340 [==============================] - 0s 75us/step - loss: 2475154.2029 - acc: 0.0000e+00 - val_loss: 1402436.9095 - val_acc: 0.0000e+00
Epoch 55/1000
340/340 [==============================] - 0s 75us/step - loss: 2315505.1176 - acc: 0.0000e+00 - val_loss: 1588566.5296 - val_acc: 0.0000e+00
Epoch 56/1000
340/340 [==============================] - 0s 76us/step - loss: 2121026.4015 - acc: 0.0000e+00 - val_loss: 1290193.4507 - val_acc: 0.0000e+00
Epoch 57/1000
340/340 [==============================] - 0s 79us/step - loss: 1822746.0500 - acc: 0.0000e+00 - val_loss: 1423201.1365 - val_acc: 0.0000e+00
Epoch 58/1000
340/340 [==============================] - 0s 76us/step - loss: 

340/340 [==============================] - 0s 71us/step - loss: 523981.7074 - acc: 0.0000e+00 - val_loss: 1037025.6316 - val_acc: 0.0000e+00
Epoch 106/1000
340/340 [==============================] - 0s 73us/step - loss: 536134.6768 - acc: 0.0000e+00 - val_loss: 554539.9951 - val_acc: 0.0000e+00
Epoch 107/1000
340/340 [==============================] - 0s 72us/step - loss: 438889.2397 - acc: 0.0000e+00 - val_loss: 613041.6916 - val_acc: 0.0000e+00
Epoch 108/1000
340/340 [==============================] - 0s 76us/step - loss: 447235.7651 - acc: 0.0000e+00 - val_loss: 556753.4161 - val_acc: 0.0000e+00
Epoch 109/1000
340/340 [==============================] - 0s 70us/step - loss: 400860.2934 - acc: 0.0029 - val_loss: 503328.3257 - val_acc: 0.0000e+00
Epoch 110/1000
340/340 [==============================] - 0s 76us/step - loss: 402319.7485 - acc: 0.0000e+00 - val_loss: 508849.5872 - val_acc: 0.0000e+00
Epoch 111/1000
340/340 [==============================] - 0s 80us/step - loss: 393412.24

340/340 [==============================] - 0s 76us/step - loss: 230145.6895 - acc: 0.0000e+00 - val_loss: 392512.2961 - val_acc: 0.0000e+00
Epoch 159/1000
340/340 [==============================] - 0s 75us/step - loss: 226818.2276 - acc: 0.0000e+00 - val_loss: 395848.7640 - val_acc: 0.0000e+00
Epoch 160/1000
340/340 [==============================] - 0s 78us/step - loss: 212237.6393 - acc: 0.0000e+00 - val_loss: 385458.3766 - val_acc: 0.0000e+00
Epoch 161/1000
340/340 [==============================] - 0s 75us/step - loss: 213366.9438 - acc: 0.0000e+00 - val_loss: 401260.1727 - val_acc: 0.0000e+00
Epoch 162/1000
340/340 [==============================] - 0s 82us/step - loss: 212601.7353 - acc: 0.0000e+00 - val_loss: 395648.2278 - val_acc: 0.0000e+00
Epoch 163/1000
340/340 [==============================] - 0s 77us/step - loss: 211794.3165 - acc: 0.0000e+00 - val_loss: 398622.3166 - val_acc: 0.0000e+00
Epoch 164/1000
340/340 [==============================] - 0s 77us/step - loss: 210853

340/340 [==============================] - 0s 73us/step - loss: 193812.7864 - acc: 0.0029 - val_loss: 407826.0921 - val_acc: 0.0000e+00
Epoch 212/1000
340/340 [==============================] - 0s 76us/step - loss: 181664.9542 - acc: 0.0000e+00 - val_loss: 454338.9145 - val_acc: 0.0000e+00
Epoch 213/1000
340/340 [==============================] - 0s 76us/step - loss: 177642.4257 - acc: 0.0000e+00 - val_loss: 421462.3754 - val_acc: 0.0000e+00
Epoch 214/1000
340/340 [==============================] - 0s 76us/step - loss: 175079.5710 - acc: 0.0000e+00 - val_loss: 379202.2253 - val_acc: 0.0000e+00
Epoch 215/1000
340/340 [==============================] - 0s 75us/step - loss: 192171.6066 - acc: 0.0000e+00 - val_loss: 445051.7451 - val_acc: 0.0000e+00
Epoch 216/1000
340/340 [==============================] - 0s 76us/step - loss: 189188.3101 - acc: 0.0000e+00 - val_loss: 442952.5037 - val_acc: 0.0000e+00
Epoch 217/1000
340/340 [==============================] - 0s 74us/step - loss: 244566.216

340/340 [==============================] - 0s 74us/step - loss: 180822.9704 - acc: 0.0000e+00 - val_loss: 386882.1863 - val_acc: 0.0000e+00
Epoch 265/1000
340/340 [==============================] - 0s 77us/step - loss: 170373.2018 - acc: 0.0029 - val_loss: 433449.7656 - val_acc: 0.0000e+00
Epoch 266/1000
340/340 [==============================] - 0s 78us/step - loss: 144206.9614 - acc: 0.0000e+00 - val_loss: 393875.1036 - val_acc: 0.0000e+00
Epoch 267/1000
340/340 [==============================] - 0s 76us/step - loss: 179351.6790 - acc: 0.0029 - val_loss: 374378.5896 - val_acc: 0.0000e+00
Epoch 268/1000
340/340 [==============================] - 0s 75us/step - loss: 128882.0039 - acc: 0.0000e+00 - val_loss: 391942.6431 - val_acc: 0.0000e+00
Epoch 269/1000
340/340 [==============================] - 0s 76us/step - loss: 139964.3993 - acc: 0.0000e+00 - val_loss: 391150.7467 - val_acc: 0.0000e+00
Epoch 270/1000
340/340 [==============================] - 0s 73us/step - loss: 162136.7316 - 

340/340 [==============================] - 0s 76us/step - loss: 142199.8794 - acc: 0.0000e+00 - val_loss: 519840.1661 - val_acc: 0.0000e+00
Epoch 318/1000
340/340 [==============================] - 0s 76us/step - loss: 128151.7249 - acc: 0.0000e+00 - val_loss: 389300.6012 - val_acc: 0.0000e+00
Epoch 319/1000
340/340 [==============================] - 0s 75us/step - loss: 117744.7631 - acc: 0.0000e+00 - val_loss: 415881.2368 - val_acc: 0.0000e+00
Epoch 320/1000
340/340 [==============================] - 0s 75us/step - loss: 133095.9689 - acc: 0.0118 - val_loss: 399681.2229 - val_acc: 0.0000e+00
Epoch 321/1000
340/340 [==============================] - 0s 76us/step - loss: 147227.4438 - acc: 0.0000e+00 - val_loss: 392558.8117 - val_acc: 0.0000e+00
Epoch 322/1000
340/340 [==============================] - 0s 76us/step - loss: 195023.2294 - acc: 0.0000e+00 - val_loss: 470705.4095 - val_acc: 0.0000e+00
Epoch 323/1000
340/340 [==============================] - 0s 75us/step - loss: 253885.177

In [45]:
y_predict = model.predict(x_test)

In [46]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 2253.252886318861


In [18]:
print(model.predict(final_predict_data))

[[52231.832]]
